In [2]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 KB 37.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install re 

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk
#for maths
import numpy as np
#for string manipulation
import string
#for importing and managing our dataset
import pandas as pd
#for pre-processing our dataset
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('twcs.csv')

In [10]:
#Specify which company we want to work with
company = "Tesco"

#Filter for answers only made by that company
answers = df.loc[df['author_id'] == company]

In [11]:
#Convert all our text to lower case
answers['text'] = answers.apply(lambda row: row['text'].lower(), axis=1)
#Strip off any trailing full stops
answers['text'] = answers.apply(lambda row: row['text'].rstrip('.'), axis=1)
#Remove any mentions to users e.g. "@johnsmith you can do this by...."
answers['text'] = answers.apply(lambda row: re.sub("\B@\w+", "", row['text']), axis=1)

In [12]:
#variable for concatinating all answers sent by the company
raw = ""

#concatinate answers into raw variable
for index, row in answers.iterrows():
    raw += ". " + row['text']

In [13]:
#download nltk assets
nltk.download('punkt')
nltk.download('wordnet')

#convert our raw sentences into sentence tokens
sentence_tokens = nltk.sent_tokenize(raw)
#convert our raw sentences into word tokens
word_tokens = nltk.word_tokenize(raw);

[nltk_data] Downloading package punkt to /home/i4624/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/i4624/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [15]:
#import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#define our function for processing a response
def response(user_response):
    #define our response variable
    robo_response=''
    #add our users input as a response
    sentence_tokens.append(user_response)
    #create out vectorizer
    vectorizer = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    #process our tokens
    diff = vectorizer.fit_transform(sentence_tokens)
    #find the similarity
    vals = cosine_similarity(diff[-1], diff)
    #select our sentence
    idx = vals.argsort()[0][-2]
    #calculate accuracy
    flat = vals.flatten()
    flat.sort()
    req_diff = flat[-2]
    if(req_diff==0):
        #if no appropriate response can be made
        robo_response=robo_response+"Sorry! I don't think I can help you with that."
        return robo_response
    else:
        #if an appropriate response is found
        robo_response = sentence_tokens[idx]        
        return robo_response

In [16]:
#define our flag to exit the loop
flag=True
#print welcome message for our chosen company
print("{companyname} Support: Welcome to {companyname} Support. I will answer your queries about {companyname}. If you wish to end the chat, type bye!".format(companyname = company))
while(flag==True):
    #get an input
    user_response = input()
    #convert to lower
    user_response=user_response.lower()
    #if they type something other than 'bye'
    if(user_response!='bye'):
        #if they show appriciation
        if(user_response=='thanks' or user_response=='thank you' ):
            #exit the loop
            flag=False
            #thank you message
            print("{companyname} Support: You are welcome.".format(companyname = company))
        else:
            #show bot is typing
            print("{companyname} Support: ".format(companyname = company), end="")
            #print our AI response
            print(response(user_response))
            sentence_tokens.remove(user_response)
    else:
        #exit the loop
        flag=False
        #exit message
        print("{companyname} Support: Thanks for chatting. I hope we could assist you today.".format(companyname = company))

Tesco Support: Welcome to Tesco Support. I will answer your queries about Tesco. If you wish to end the chat, type bye!
Tesco Support: so we can support you further.
Tesco Support: after i have these i will get that refund out to you.
Tesco Support: happy eating and enjoy.
Tesco Support: Sorry! I don't think I can help you with that.
Tesco Support: christmas markets?
Tesco Support: christmas markets?
Tesco Support: Sorry! I don't think I can help you with that.
Tesco Support: Sorry! I don't think I can help you with that.
Tesco Support: You are welcome.
